In [1]:
!pip3 install virtualenv
!virtualenv mlcv
!source /content/mlcv/bin/activate;
!sudo apt-get update -y

!sudo apt-get install python3.7
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 25.5 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 347ms
  creator CPython3Posix(dest=/content/mlcv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.0, setuptools==69.5.1, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [830 kB]
Hit:5 

In [2]:
!python --version

Python 3.7.17


# Section 1: TCAV
Please refer this source https://github.com/tensorflow/tcav on how you can import and run TCAV. The repository consists of an example (https://github.com/tensorflow/tcav/blob/master/Run_TCAV_on_colab.ipynb) of TCAV scores using GoogLeNet model trained on ImageNet dataset. The examples demonstrated the results based the concepts ["dotted","striped","zigzagged"] with target being 'zebra'.

You are allowed to change the code template in accordance with the example provided.

In [3]:
!sudo apt install python3-pip
!sudo apt install python3.7-distutils

!pip3 install tensorflow==1.15
!pip install keras==2.2.4

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python3-pip python3-setuptools python3-wheel
0 upgraded, 3 newly installed, 0 to remove and 46 not upgraded.
Need to get 1,677 kB of archives.
After this operation, 8,967 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-setuptools all 59.6.0-1.2ubuntu0.22.04.1 [339 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-wheel all 0.37.1-2ubuntu0.22.04.1 [32.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip all 22.0.2+dfsg-1ubuntu0.4 [1,305 kB]
Fetched 1,677 kB in 1s (2,247 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog

In [4]:
!git clone https://github.com/pnxenopoulos/cav-keras.git
%cd cav-keras

Cloning into 'cav-keras'...
remote: Enumerating objects: 369, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 369 (delta 1), reused 0 (delta 0), pack-reused 363
Receiving objects: 100% (369/369), 23.58 MiB | 25.74 MiB/s, done.
Resolving deltas: 100% (161/161), done.
/content/cav-keras


In [5]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from keras.datasets import cifar100, cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50

import sys
import os
sys.path.insert(0, os.path.abspath('../..'))

from cav.tcav import *

np.random.seed(1996)

In [6]:
def extract_airplane_set():
    #TODO: Extract the indexes of all the images with 'airplane' as the output label from both train and test.
    #TODO: Return the preprocessed sets of both train and test sets
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    a_indexes_train = np.where(y_train == 0)[0]
    b_indexes_train = np.where(y_train == 2)[0]
    a_indexes_test = np.where(y_test == 0)[0]
    b_indexes_test = np.where(y_test == 2)[0]

    # Combine airplane and bird indexes
    train_indexes = np.concatenate((a_indexes_train, b_indexes_train))
    test_indexes = np.concatenate((a_indexes_test, b_indexes_test))

    # Select data for training and testing
    x_train_ab = x_train[train_indexes]
    y_train_ab = y_train[train_indexes]
    x_test_ab = x_test[test_indexes]
    y_test_ab = y_test[test_indexes]

    y_train_ab = (y_train_ab == 0).astype(int)
    y_test_ab = (y_test_ab == 0).astype(int)

    # Normalize data
    x_train_ab = x_train_ab.astype('float32') / 255.0
    x_test_ab = x_test_ab.astype('float32') / 255.0

    return x_train_ab, y_train_ab, x_test_ab, y_test_ab

def extract_cloud_set():
    (x_train_concept, y_train_concept), (x_test_concept, y_test_concept) = cifar100.load_data()
    #TODO: Extract the indexes of all the images with 'cloud' as the output label for ONLY train set.
    #TODO: Return ONLY train set (No preprocess needed)
    cloud_indexes = np.where(y_train_concept.flatten() == 23)[0]
    x_train_concept_cloud = x_train_concept[cloud_indexes]
    y_train_concept_cloud = y_train_concept[cloud_indexes]

    return x_train_concept_cloud, y_train_concept_cloud

def load_and_transfer_learn():
    # Load the ResNet50 model without the top layers
    resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape=(32, 32, 3), pooling=None)

    # Freeze the pre-trained layers
    for layer in resnet_model.layers:
        layer.trainable = False

    # Add custom top layers
    model = Sequential()
    model.add(resnet_model)
    model.add(GlobalAveragePooling2D())

    layers = [
        Dense(1024, activation='relu'), Dropout(0.35),
        Dense(512, activation='relu'), Dropout(0.25),
        Dense(256, activation='relu'), Dropout(0.2),
        Dense(128, activation='relu'), Dropout(0.175),
        Dense(64, activation='relu'), Dropout(0.125),
        Dense(1, activation='sigmoid')
    ]

    for layer in layers:
        model.add(layer)

    return model

def tcav_scores(model, x_train, y_train, x_train_concept, bottleneck):
    tf.compat.v1.disable_eager_execution()
    tcav_obj = TCAV()
    tcav_obj.set_model(model)
    tcav_obj.split_model(bottleneck=bottleneck, conv_layer=True)
    tcav_obj.train_cav(x_train_concept)
    tcav_obj.calculate_sensitivity(x_train, y_train)
    tcav_obj.print_sensitivity()

In [7]:
x_train, y_train, x_test, y_test = extract_airplane_set()

170498071/170498071 [==============================] - 4s 0us/step


In [8]:
x_train_concept, y_train_concept = extract_cloud_set()

169001437/169001437 [==============================] - 4s 0us/step


In [45]:
model = load_and_transfer_learn()

##TODO: Define your choice of batch_size, number of epochs, loss function (e.g. categorical crossentropy), and optimizer (SGD\Adam)
batch_size = 32
epochs = 5
loss_func = 'binary_crossentropy'    #<--- has to be in string format
optim = 'adam'       #<--- has to be in string format

model.compile(loss=loss_func, optimizer=optim, metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True)

Train on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 5s 486us/sample - loss: 0.7147 - accuracy: 0.5007
Epoch 2/5
10000/10000 [==============================] - 3s 330us/sample - loss: 0.6949 - accuracy: 0.5008
Epoch 3/5
10000/10000 [==============================] - 3s 336us/sample - loss: 0.6896 - accuracy: 0.5248
Epoch 4/5
10000/10000 [==============================] - 3s 327us/sample - loss: 0.6616 - accuracy: 0.5875
Epoch 5/5
10000/10000 [==============================] - 3s 330us/sample - loss: 0.6459 - accuracy: 0.6202


In [46]:
model.summary()

Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 global_average_pooling2d_8  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_48 (Dense)            (None, 1024)              2098176   
                                                                 
 dropout_40 (Dropout)        (None, 1024)              0         
                                                                 
 dense_49 (Dense)            (None, 512)               524800    
                                                                 
 dropout_41 (Dropout)        (None, 512)               0         
                                                     

In [47]:
tcav_scores(model, x_train, y_train, x_train_concept, bottleneck=5)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


The sensitivity of class 1 is  0.1094
The sensitivity of class 0 is  0.9086


In [48]:
tcav_scores(model, x_train, y_train, x_train_concept, bottleneck=7)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


The sensitivity of class 1 is  0.7232
The sensitivity of class 0 is  0.74


In [49]:
tcav_scores(model, x_train, y_train, x_train_concept, bottleneck=11)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


The sensitivity of class 1 is  1.0
The sensitivity of class 0 is  0.0
